# Final Project: AI Agents for Lifelong Learning  

## The Problem  

Lifelong learning depends not just on access to information, but on consistent exposure to **relevant, up-to-date content** that aligns with individual interests and contributes meaningfully to personal and professional growth.  

While information is abundant, there’s no solution that effectively curates **up-to-date, relevant content** tailored to evolving interests.  

I’ve faced this pain point firsthand. Staying current with newsletters, research updates, and niche discussions is overwhelming and inefficient. 

This project addresses a critical gap: leveraging the constant flow of new content to accelerate learning in a way that is personalized, actionable, and aligned with long-term growth.  



## The Solution  

This project introduces a **multi-agent system** designed to act as a learning companion, curating and delivering content that supports continuous, personalized education. Using **Crew AI**, we build a system that:  

1. **Aggregates Content**: Dynamically retrieves articles, memos, and updates from diverse, high-quality sources like Hacker News and professional newsletters.  
2. **Filters for Relevance**: Identifies the most pertinent content aligned with user-defined learning goals and interests.  
3. **Summarizes for Precision**: Provides concise overviews, enabling users to quickly assess the value of each piece while linking to the original material for deeper exploration.  

## How It Works  

The system leverages the synergy of **AI agents** and **Large Language Models (LLMs)** to ensure relevance and adaptability:  

- **Fetcher Agent**: Collects content from predefined sources, ensuring a consistent and diverse stream of information.  
- **Filter Agent**: Evaluates and prioritizes content based on relevance to user preferences and learning goals.  
- **Summarizer Agent**: Distills selected content into concise summaries, retaining key insights for quick and effective understanding.  
- **Notifier Agent**: Logs curated content and sends summaries to the user via WhatsApp for seamless accessibility.  


## Impact on Lifelong Learning  

This system supports lifelong learning by:  
1. **Promoting Focus**: Reducing noise and information overload by prioritizing relevant content.  
2. **Encouraging Exploration**: Suggesting diverse sources and materials tailored to specific interests.  
3. **Adapting Over Time**: Growing alongside the user’s learning journey, offering increasingly precise recommendations.  

## Evaluation Metrics  

The success of this project will be measured by:  
1. **Relevance Accuracy**: At least 90% of recommendations align with user interests.  
2. **Engagement**: Users interact with >80% of delivered content through summaries or full articles.  
3. **Feedback Adaptability**: The system demonstrates a measurable improvement in relevance precision over time.  

---

## Final Goal  

To create a precise, adaptable, and user-focused system that empowers lifelong learners by seamlessly integrating relevant, high-quality content into their daily routines.  

This project emphasizes the transformative power of AI in shaping learning habits, demonstrating that with the right tools, lifelong learning can become not just a goal, but a natural and effortless part of life.  

---

### Repository  
[ai-agents-crewAI](https://github.com/felipebpl/ai-agents-crewAI)  


In [15]:
import requests
import logging
import os
from dotenv import load_dotenv
from typing import List, Dict
from bs4 import BeautifulSoup

# Load environment variables from .env file
load_dotenv()

# Validate environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Missing 'OPENAI_API_KEY' in environment variables.")

# WHATSAPP_API_TOKEN = os.getenv("WHATSAPP_API_TOKEN")
# USER_PHONE = os.getenv("USER_PHONE")

# if not WHATSAPP_API_TOKEN or not USER_PHONE:
#     raise ValueError("Missing 'WHATSAPP_API_TOKEN' or 'USER_PHONE' in environment variables.")

OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"
WHATSAPP_API_URL = "https://your-whatsapp-api-endpoint"

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')

# Parse html content
def preprocess_html_to_text(html_content: str) -> str:
    """
    Preprocesses HTML content by extracting clean text for AI summarization.

    Args:
        html_content (str): Raw HTML content.

    Returns:
        str: Cleaned and readable plain text.
    """
    # Parse HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract and join visible text
    clean_text = ' '.join(soup.stripped_strings)

    return clean_text


# Utility function for OpenAI API
def generate_summary(prompt: str, text: str) -> str:
    cleaned_text = preprocess_html_to_text(text)
    
    headers = {"Authorization": f"Bearer {OPENAI_API_KEY}", "Content-Type": "application/json"}
    payload = {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": cleaned_text}
        ],
        "max_tokens": 150,
        "temperature": 0.7
    }
    response = requests.post(OPENAI_API_URL, headers=headers, json=payload)
    response.raise_for_status()
    response_data = response.json()
    return response_data['choices'][0]['message']['content']


# Agents
class FetcherAgent:
    def __init__(self, sources: List[str]):
        self.sources = sources

    def fetch_content(self) -> List[Dict[str, str]]:
        logging.info("Fetching content from sources.")
        results = []
        for source in self.sources:
            try:
                logging.info(f"Scraping content from: {source}")
                response = requests.get(source)
                response.raise_for_status()
                results.append({"source": source, "content": response.text})
            except Exception as e:
                logging.error(f"Error scraping {source}: {e}")
        return results


class FilterAgent:
    def __init__(self, keywords: List[str]):
        self.keywords = keywords

    def filter_content(self, fetched_content: List[Dict[str, str]]) -> List[Dict[str, str]]:
        logging.info("Filtering content based on keywords.")
        filtered_results = [
            item for item in fetched_content
            if any(keyword.lower() in item["content"].lower() for keyword in self.keywords)
        ]
        logging.info(f"Filtered {len(filtered_results)} items.")
        return filtered_results


class SummarizerAgent:
    def __init__(self, prompt: str):
        self.prompt = prompt

    def summarize_content(self, filtered_content: List[Dict[str, str]]) -> List[Dict[str, str]]:
        logging.info("Summarizing content.")
        summarized_results = []
        for item in filtered_content:
            try:
                summary = generate_summary(self.prompt, item["content"])
                summarized_results.append({"source": item["source"], "summary": summary})
            except Exception as e:
                logging.error(f"Error summarizing content from {item['source']}: {e}")
        return summarized_results


class NotifierAgent:
    def __init__(self):
        self.api_url = WHATSAPP_API_URL
        self.api_token = WHATSAPP_API_TOKEN
        self.user_phone = USER_PHONE

    def send_notifications(self, summaries: List[Dict[str, str]]):
        logging.info("Logging and sending notifications.")
        for summary in summaries:
            message = f"Source: {summary['source']}\nSummary:\n{summary['summary']}"
            logging.info(message)
            self._send_whatsapp_message(message)

    def _send_whatsapp_message(self, message: str):
        payload = {
            "to": self.user_phone,
            "type": "text",
            "text": {"body": message}
        }
        headers = {
            "Authorization": f"Bearer {self.api_token}",
            "Content-Type": "application/json"
        }
        try:
            response = requests.post(self.api_url, json=payload, headers=headers)
            response.raise_for_status()
            logging.info("Message sent successfully.")
        except Exception as e:
            logging.error(f"Error sending message: {e}")

# Main Execution
if __name__ == "__main__":
    # Define user preferences
    sources = [
        "https://news.ycombinator.com/",
        "https://www.paulgraham.com/articles.html"
    ]
    keywords = ["AI", "machine learning", "venture capital"]
    prompt = """
    You are a specialized assistant for a busy founder. Your goal is to distill the essence of the following text into a concise and actionable summary. Focus on extracting the most relevant and insightful points that contribute to strategic decision-making, personal growth, or learning essential trends. Avoid fluff or generalities. Write simply and clearly, as if explaining to a peer.

    Use the following structure:
    - **Title**: Source title.
    - **Key Takeaway**: One-sentence summary.
    - **Why It Matters**: Brief significance.
    - **Details**: Bullet points with essential context.

    Text:
    """

    # Initialize agents
    fetcher = FetcherAgent(sources)
    filter_agent = FilterAgent(keywords)
    summarizer = SummarizerAgent(prompt)
    # notifier = NotifierAgent()

    # Run pipeline
    fetched_content = fetcher.fetch_content()
    filtered_content = filter_agent.filter_content(fetched_content)
    summarized_content = summarizer.summarize_content(filtered_content)
    # notifier.send_notifications(summarized_content)
    print(summarized_content)


[{'source': 'https://news.ycombinator.com/', 'summary': "- **Title**: Hacker News Overview\n- **Key Takeaway**: Hacker News posts cover a variety of topics, including technology, programming, science, and current events, with user engagement varying across posts.\n- **Why It Matters**: Staying updated on these discussions can provide insights into trending topics, technological developments, and potential business opportunities.\n- **Details**: \n   - Erlang hot code updates, Open Riak, and Python's object-oriented programming are among the trending programming topics.\n   - Discussions on artificial intelligence breakthroughs highlight the importance of deep learning.\n   - There's a user-created black hole simulation for iPhone, indicating interest in astrophysics and app development.\n   - Questions about traditional coding practices, like the 80 character line"}, {'source': 'https://www.paulgraham.com/articles.html', 'summary': '- **Title**: Various Essays and Articles\n- **Key Tak